In [1]:
import pandas as pd 
import numpy as np
import openpyxl
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)

In [2]:
metro = pd.read_csv('eco2mix-metropoles-tr.csv', sep=';')
mfdata = pd.read_csv('meteo-france-donneesq-covid19.csv', sep =';', encoding = 'latin1', on_bad_lines='skip', decimal=',')
dpe = pd.read_excel('DPE.xlsx')

C:\Users\alexi\AppData\Local\Temp\ipykernel_8880\4124632932.py:1: DtypeWarning: Columns (7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  metro = pd.read_csv('eco2mix-metropoles-tr.csv', sep=';')


# Météo France : MFDATA

In [3]:
mfdata['Date'] = pd.to_datetime(mfdata['DATE'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
mfdata.drop(columns=['DATE', 'POSTE', 'POSTE', 'LON', 'LAT', 'ALT', 'QRR', 'QTN', 'QTX', 'QTM', 'QFFM', 'QUN', 'QUX', 
                         'QDHUMI40', 'QDHUMI80', 'QUM'], inplace=True)                      

In [4]:
mfdata = mfdata.rename(columns={'RR' : 'Precipitation', 
                                      'TN' : 'Temperature_minimale',
                                      'TX' : 'Temperature_maximale', 
                                      'TM' : 'Temperature_moyenne',
                                      'FFM' : 'Moyenne_du_vent_a_10m', 
                                      'UN' : 'Humidite_relative_minimale',
                                      'UX' : 'Humidite_relative_maximale',
                                      'DHUMI40' : 'Duree_humidite_inf_a_40_pct', 
                                      'DHUMI80' : 'Duree_humidite_sup_a_80_pct',
                                      'UM' : 'Humidite_moyenne',
                                      'NOM' : 'Ville'})

# Consommation par métropole : METRO

In [5]:
metro['Ville']=''
metro.loc[metro['Métropole'] == 'Clermont Auvergne Métropole', 'Ville'] = 'CLERMONT-FD'
metro.loc[metro['Métropole'] == "Métropole d'Aix-Marseille-Provence", 'Ville'] = 'MARSEILLE'
metro.loc[metro['Métropole'] == 'Métropole de Lyon', 'Ville'] = 'LYON-BRON'
metro.loc[metro['Métropole'] == 'Dijon Métropole', 'Ville'] = 'DIJON-LONGVIC'
metro.loc[metro['Métropole'] == 'Métropole Grenoble-Alpes-Métropole', 'Ville'] = 'GRENOBLE-ST GEOIRS'
metro.loc[metro['Métropole'] == 'Métropole Européenne de Lille', 'Ville'] = 'LILLE-LESQUIN'
metro.loc[metro['Métropole'] == 'Métropole du Grand Nancy', 'Ville'] = 'NANCY-ESSEY'
metro.loc[metro['Métropole'] == 'Montpellier Méditerranée Métropole', 'Ville'] = 'MONTPELLIER-AEROPORT'
metro.loc[metro['Métropole'] == 'Nantes Métropole', 'Ville'] = 'NANTES-BOUGUENAIS' #Non
metro.loc[metro['Métropole'] == 'Bordeaux Métropole', 'Ville'] = 'BORDEAUX-MERIGNAC' #Non
metro.loc[metro['Métropole'] == 'Brest Métropole', 'Ville'] = 'BREST-GUIPAVAS' #Non
metro.loc[metro['Métropole'] == "Métropole Nice Côte d'Azur", 'Ville'] = 'NICE' #vérif si différent de marseilles et hyeres
metro.loc[metro['Métropole'] == "Métropole du Grand Paris", 'Ville'] = 'PARIS-MONTSOURIS'
metro.loc[metro['Métropole'] == "Eurométropole de Strasbourg", 'Ville'] = 'STRASBOURG-ENTZHEIM'
metro.loc[metro['Métropole'] == "Saint-Etienne Métropole", 'Ville'] = 'ST ETIENNE-BOUTHEON' #vérifier si différent de clermond
metro.loc[metro['Métropole'] == "Métropole Rennes Métropole", 'Ville'] = 'RENNES-ST JACQUES'#Non
metro.loc[metro['Métropole'] == "Métropole Rouen Normandie", 'Ville'] = 'ROUEN-BOOS'
metro.loc[metro['Métropole'] == "Métropole Toulon-Provence-Méditerranée", 'Ville'] = 'HYERES'#vérifier si différent de marseilles et nice
metro.loc[metro['Métropole'] == "Toulouse Métropole", 'Ville'] = 'TOULOUSE-BLAGNAC'
metro.loc[metro['Métropole'] == "Orléans Métropole", 'Ville'] = 'ORLEANS'
metro.loc[metro['Métropole'] == "Tours Métropole Val de Loire", 'Ville'] = 'TOURS'

#nom métropole = station météo

In [6]:
metro.dropna(subset='Consommation (MW)', inplace=True)

In [7]:
metro['Date'] = pd.to_datetime(metro['Date'])
metro_mean = metro.groupby(["Ville", pd.Grouper(key="Date", freq="D")]).agg({"Consommation (MW)": "mean"}).reset_index()

In [8]:
metro_mean['Date'] = metro_mean['Date'].dt.strftime('%Y-%m-%d') 
df = pd.merge(metro_mean, mfdata, on=['Date', 'Ville'], how='left')

# Préparation pour la régression

In [9]:
df.dropna(inplace=True)

In [10]:
cols = ['Precipitation',
       'Temperature_minimale', 'Temperature_maximale', 'Temperature_moyenne',
       'Moyenne_du_vent_a_10m', 'Humidite_relative_minimale',
       'Humidite_relative_maximale', 'Duree_humidite_inf_a_40_pct',
       'Duree_humidite_sup_a_80_pct', 'Humidite_moyenne']

### F-TEST
Le F-test mesure le degré de dépendance linéaire entre les variables et la target via l'analyse de la variance (ANOVA). Il s'agit de comparer la variance de la target avec en la régressant sur les variables, à la variance de la target sans la régression.

In [11]:
#F-test : 
from sklearn.feature_selection import SelectKBest, f_regression
import numpy as np
#selection with Ftest

X_fscore = df[cols].astype(float).values
y_fscore = df["Consommation (MW)"].astype(float).values

selector = SelectKBest(f_regression, k=2)
selector.fit(X_fscore, y_fscore)

# F-test scores of all the variables
scores = selector.scores_

# Sort the variables by their F-test scores
sorted_indices = np.argsort(scores)[::-1]
sorted_cols = [cols[i] for i in sorted_indices]
print("Le F-score des variables est :")

for col in sorted_cols:
    print(f"{col}: {scores[cols.index(col)]}")

Le F-score des variables est :
Humidite_relative_maximale: 153.92320503183745
Temperature_maximale: 109.71490662174033
Temperature_moyenne: 73.25026621247353
Moyenne_du_vent_a_10m: 72.38693329884752
Temperature_minimale: 13.751447988906051
Duree_humidite_sup_a_80_pct: 13.656740891348132
Humidite_moyenne: 9.048455213730026
Humidite_relative_minimale: 5.586829425322859
Precipitation: 0.00670069986456052
Duree_humidite_inf_a_40_pct: 0.0016681647662180011


### RFE
Le RFE procède par itération en éliminant les variables les moins importantes (ici selon le R²), et chaque métrique avec le score le plus faible est éliminée 

In [12]:
#RFE score
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

X_rfe = df[cols].astype(float).values
y_rfe = df["Consommation (MW)"].astype(float).values

# Initialize the model
model = LinearRegression()

# Initialize the RFECV object with the model and cross-validation parameters
selector = RFECV(estimator=model, step=1, cv=5, scoring='r2')

# Fit the RFECV object to the data
selector.fit(X_rfe, y_rfe)

# RFE ranking of the variables
ranks = selector.ranking_

# Sort the variables by their RFE ranking
sorted_indices = np.argsort(ranks)
sorted_cols = [cols[i] for i in sorted_indices]
print("Le classement RFE des variables est :")
for col in sorted_cols:
    print(f"{col}: {ranks[cols.index(col)]}")

#lowest RFE ranking is the most important variable

Le classement RFE des variables est :
Moyenne_du_vent_a_10m: 1
Temperature_maximale: 2
Temperature_minimale: 3
Humidite_relative_maximale: 4
Temperature_moyenne: 5
Precipitation: 6
Humidite_moyenne: 7
Humidite_relative_minimale: 8
Duree_humidite_inf_a_40_pct: 9
Duree_humidite_sup_a_80_pct: 10


# Régression préliminaire : Consommation sur DPE

## Données

In [13]:
df_aggregated = df.groupby(['Ville'])['Consommation (MW)'].sum().reset_index()
df_dpe = pd.merge(df_aggregated, dpe, on='Ville', how='left')

In [14]:
df_dpe['D-E'] = df_dpe['D'] + df_dpe['E']
df_dpe.rename(columns={"%F&G ensemble" : "F-G"}, inplace=True)
df_dpe.drop(columns=['D', 'E', 'F', 'G'], inplace=True)

## Score variables explicatives : F-test et RFE

In [15]:
cols = ['Nombre de DPE', 'A-B-C', 'Nombre de DPE F ou G', 'F-G', 'D-E']

#F-test : 
from sklearn.feature_selection import SelectKBest, f_regression
import numpy as np
#selection with Ftest

X_fscore = df_dpe[cols].astype(float).values
y_fscore = df_dpe["Consommation (MW)"].astype(float).values

selector = SelectKBest(f_regression, k=2)
selector.fit(X_fscore, y_fscore)

# F-test scores of all the variables
scores = selector.scores_

# Sort the variables by their F-test scores
sorted_indices = np.argsort(scores)[::-1]
sorted_cols = [cols[i] for i in sorted_indices]

print("Le F-score des variables est :")
for col in sorted_cols:
    print(f"{col}: {scores[cols.index(col)]}")

print("------------------------------")

#RFE score
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

X_rfe = df_dpe[cols].astype(float).values
y_rfe = df_dpe["Consommation (MW)"].astype(float).values

# Initialize the model
model = LinearRegression()

# Initialize the RFECV object with the model and cross-validation parameters
selector = RFECV(estimator=model, step=1, cv=5, scoring='r2')

# Fit the RFECV object to the data
selector.fit(X_rfe, y_rfe)

# RFE ranking of the variables
ranks = selector.ranking_

# Sort the variables by their RFE ranking
sorted_indices = np.argsort(ranks)
sorted_cols = [cols[i] for i in sorted_indices]
print("Le classement RFE des variables est :")
for col in sorted_cols:
    print(f"{col}: {ranks[cols.index(col)]}")

#lowest RFE ranking is the most important variable

Le F-score des variables est :
Nombre de DPE: 175.15082818244815
Nombre de DPE F ou G: 170.45579656398348
F-G: 15.156146786048824
A-B-C: 1.9632730847982787
D-E: 1.1698612409989928
------------------------------
Le classement RFE des variables est :
A-B-C: 1
Nombre de DPE F ou G: 1
F-G: 1
D-E: 1
Nombre de DPE: 2


In [16]:
df_dpe = df_dpe.reindex(columns=['Ville', 'Consommation (MW)', 'A-B-C', 'D-E', 'F-G', 'Nombre de DPE F ou G', 'Nombre de DPE'])

In [17]:
df_dpe

,Ville,Consommation (MW),A-B-C,D-E,F-G,Nombre de DPE F ou G,Nombre de DPE
0,BORDEAUX-MERIGNAC,2.668644e+05,0.266,0.585,0.149,617,4129
1,BREST-GUIPAVAS,5.487575e+04,0.261,0.620,0.120,216,1804
2,CLERMONT-FD,7.900606e+04,0.159,0.661,0.180,322,1786
3,DIJON-LONGVIC,6.532091e+04,0.306,0.532,0.161,457,2831
4,GRENOBLE-ST GEOIRS,2.140559e+05,0.377,0.484,0.139,327,2352
5,HYERES,9.723210e+04,0.332,0.563,0.104,164,1570
6,LILLE-LESQUIN,4.483581e+05,0.143,0.666,0.190,691,3633
7,LYON-BRON,4.718642e+05,0.328,0.541,0.131,1325,10107
8,MARSEILLE,7.502686e+05,0.400,0.524,0.076,674,8917
9,MONTPELLIER-AEROPORT,1.602625e+05,0.463,0.461,0.077,320,4146


## Régression sans le nombre de DPE

In [18]:
drop_dpe_ = ['Ville', 'Consommation (MW)', 'A-B-C', 'Nombre de DPE']

In [19]:
X_dpe = df_dpe.drop(drop_dpe_, axis=1).astype(float).values
y_dpe = df_dpe["Consommation (MW)"].astype(float).values

In [20]:
import statsmodels.api as sm

# Ajouter une colonne de 1 pour l'ordonnée à l'origine
X_dpe = sm.add_constant(X_dpe)

# Instancier le modèle de régression linéaire avec statsmodels
model = sm.OLS(y_dpe, X_dpe)

# Entraîner le modèle sur les données
results = model.fit()

# Créer un tableau de résultats avec les coefficients, les écart-types et les p-values
coef_table = pd.DataFrame({
    "Variable": ["Intercept"] + list(df_dpe.drop(drop_dpe_, axis=1).columns),
    "Coefficient": results.params,
    "Ecart-type": results.bse,
    "P-value": results.pvalues
}).set_index("Variable")
# Afficher le tableau de résultats
print(coef_table)


                       Coefficient     Ecart-type       P-value
Variable                                                       
Intercept             2.635522e+05  291031.357296  3.778144e-01
D-E                  -9.629552e+02  558787.435511  9.986451e-01
F-G                  -1.257715e+06  805801.885180  1.369871e-01
Nombre de DPE F ou G  2.562142e+02      30.442280  1.816831e-07


## Régression avec nombre de DPE

In [21]:
drop_dpe = ['Ville', 'Consommation (MW)', 'A-B-C']#, 'Nombre de DPE']

In [22]:
X_dpe = df_dpe.drop(drop_dpe, axis=1).astype(float).values
y_dpe = df_dpe["Consommation (MW)"].astype(float).values

In [23]:
model = sm.OLS(y_dpe, X_dpe)

# Entraîner le modèle sur les données
results = model.fit()

# Créer un tableau de résultats avec les coefficients, les écart-types et les p-values
coef_table = pd.DataFrame({
    "Variable": ["Intercept"] + list(df_dpe.drop(drop_dpe_, axis=1).columns),
    "Coefficient": results.params,
    "Ecart-type": results.bse,
    "P-value": results.pvalues
}).set_index("Variable")
# Afficher le tableau de résultats
print(coef_table)


                        Coefficient     Ecart-type   P-value
Variable                                                    
Intercept             -75797.826707  240468.642636  0.756439
D-E                   191908.896077  844397.015481  0.822922
F-G                      103.652850      58.902133  0.096432
Nombre de DPE F ou G      47.171400      15.780217  0.008241


# Régression totale

In [24]:
df

,Ville,Date,Consommation (MW),Precipitation,Temperature_minimale,Temperature_maximale,Temperature_moyenne,Moyenne_du_vent_a_10m,Humidite_relative_minimale,Humidite_relative_maximale,Duree_humidite_inf_a_40_pct,Duree_humidite_sup_a_80_pct,Humidite_moyenne
0,BORDEAUX-MERIGNAC,2020-01-01,616.536842,0.0,7.5,11.2,9.9,2.6,80.0,94.0,0.0,1440.0,88.0
1,BORDEAUX-MERIGNAC,2020-01-02,669.416667,0.0,6.2,11.0,8.3,2.3,70.0,96.0,0.0,1007.0,84.0
2,BORDEAUX-MERIGNAC,2020-01-03,739.812500,1.6,4.4,13.0,9.2,3.0,86.0,100.0,0.0,1440.0,95.0
3,BORDEAUX-MERIGNAC,2020-01-04,685.916667,0.0,6.0,10.3,7.2,2.1,66.0,100.0,0.0,1207.0,91.0
4,BORDEAUX-MERIGNAC,2020-01-05,714.541667,0.0,-0.1,9.2,3.6,1.6,60.0,100.0,0.0,988.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9882,TOURS,2021-04-17,204.010526,0.0,6.2,13.5,9.0,3.8,40.0,76.0,1.0,0.0,58.0
9883,TOURS,2021-04-18,190.021053,0.0,1.6,14.9,8.9,2.2,36.0,88.0,113.0,303.0,64.0
9884,TOURS,2021-04-19,203.431579,0.0,2.1,18.6,11.1,1.5,31.0,93.0,268.0,383.0,61.0
9885,TOURS,2021-04-20,192.084211,0.0,8.8,18.0,12.9,2.4,36.0,76.0,83.0,0.0,58.0


In [25]:
import statsmodels.api as sm

# Sélectionner les colonnes pour la régression linéaire
col_infos = df[['Date', 'Ville']]
drop_cols_tot = ['Consommation (MW)', 'Date', 'Ville']
X = df.drop(drop_cols_tot, axis=1).astype(float).values
y = df["Consommation (MW)"].astype(float).values

# Ajouter une colonne de 1 pour l'ordonnée à l'origine
X = sm.add_constant(X)

# Instancier le modèle de régression linéaire avec statsmodels
model = sm.OLS(y, X)

# Entraîner le modèle sur les données
results = model.fit()

# Créer un tableau de résultats avec les coefficients, les écart-types et les p-values
coef_table = pd.DataFrame({
    "Variable": ["Intercept"] + list(df.drop(drop_cols_tot, axis=1).columns),
    "Coefficient": results.params,
    "Ecart-type": results.bse,
    "P-value": results.pvalues
}).set_index("Variable")

# Afficher le tableau de résultats
print(coef_table)

                             Coefficient  Ecart-type       P-value
Variable                                                          
Intercept                    3973.455554  213.997070  1.143033e-75
Precipitation                   7.910661    2.122108  1.942918e-04
Temperature_minimale           56.800415    6.264022  1.447935e-19
Temperature_maximale          -39.702650    8.919972  8.641968e-06
Temperature_moyenne           -30.627303   13.743698  2.587228e-02
Moyenne_du_vent_a_10m        -109.526824    5.584804  4.913590e-84
Humidite_relative_minimale      4.324918    1.894626  2.246755e-02
Humidite_relative_maximale    -13.702296    2.135974  1.472988e-10
Duree_humidite_inf_a_40_pct    -0.208599    0.092886  2.474184e-02
Duree_humidite_sup_a_80_pct     0.133709    0.066669  4.493070e-02
Humidite_moyenne              -19.019574    4.020321  2.266871e-06


# Régression sur température, vent et humidité

Pour rappel : 
- F-Test 
    - Humidite_relative_maximale: 153.92320503183745
    - Temperature_maximale: 109.71490662174033
    - Temperature_moyenne: 73.25026621247353
    - Moyenne_du_vent_a_10m: 72.38693329884752
    - Temperature_minimale: 13.751447988906051
    - Duree_humidite_sup_a_80_pct: 13.656740891348132
    - Humidite_moyenne: 9.048455213730026
    - Humidite_relative_minimale: 5.586829425322859
    - Precipitation: 0.00670069986456052
    - Duree_humidite_inf_a_40_pct: 0.0016681647662180011
- Rfe : 
    - Moyenne_du_vent_a_10m: 1
    - Temperature_maximale: 2
    - Temperature_minimale: 3
    - Humidite_relative_maximale: 4
    - Temperature_moyenne: 5
    - Precipitation: 6
    - Humidite_moyenne: 7
    - Humidite_relative_minimale: 8
    - Duree_humidite_inf_a_40_pct: 9
    - Duree_humidite_sup_a_80_pct: 10

Les variables qu ressortent le plus souvent sont : humidité max, vent et temp max

In [26]:
visual_cols = ['Consommation (MW)','Temperature_maximale', 'Moyenne_du_vent_a_10m', 'Humidite_relative_maximale']
df[visual_cols]

,Consommation (MW),Temperature_maximale,Moyenne_du_vent_a_10m,Humidite_relative_maximale
0,616.536842,11.2,2.6,94.0
1,669.416667,11.0,2.3,96.0
2,739.812500,13.0,3.0,100.0
3,685.916667,10.3,2.1,100.0
4,714.541667,9.2,1.6,100.0
...,...,...,...,...
9882,204.010526,13.5,3.8,76.0
9883,190.021053,14.9,2.2,88.0
9884,203.431579,18.6,1.5,93.0
9885,192.084211,18.0,2.4,76.0


In [27]:
import statsmodels.api as sm

# Définir les colonnes à inclure dans la régression linéaire
selected_cols = ['Temperature_maximale', 'Moyenne_du_vent_a_10m', 'Humidite_relative_maximale']
X = df[selected_cols].astype(float).values 
y = df["Consommation (MW)"].astype(float).values

# Ajouter une colonne de 1 pour l'ordonnée à l'origine
X = sm.add_constant(X)

# Instancier le modèle de régression linéaire avec statsmodels
model = sm.OLS(y, X)

# Entraîner le modèle sur les données
results = model.fit()

# Créer un tableau de résultats avec les coefficients, les écart-types et les p-values
coef_table = pd.DataFrame({
    "Variable": ["Intercept"] + selected_cols,
    "Coefficient": results.params,
    "Ecart-type": results.bse,
    "P-value": results.pvalues
}).set_index("Variable")

# Afficher le tableau de résultats
print(coef_table)


                            Coefficient  Ecart-type        P-value
Variable                                                          
Intercept                   3109.904413  109.458869  1.002773e-170
Temperature_maximale         -23.096964    1.269682   9.384796e-73
Moyenne_du_vent_a_10m        -76.306383    5.169848   8.819478e-49
Humidite_relative_maximale   -19.914025    1.034212   3.926522e-81
